# Imports

In [1]:
import math
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image

import tensorflow as tf
from tensorflow.compat.v1.saved_model import load, tag_constants

%matplotlib inline

# Constant

In [2]:
ALPHABETS = {
    0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G', 7:'H', 8:'I',9:'J',
    10:'K', 11:'L', 12:'M', 13:'N', 14:'O', 15:'P', 16:'Q', 17:'R', 18:'S',
    19:'T', 20:'U', 21:'V', 22:'W', 23:'X', 24: 'Y', 25:'Z'
}

SAMPLE_IMAGE = 'Image_Samples/Sample1.png'

ARR_LEFT_SECTION_CONTOURS = []
ARR_PAYEE_CONTOURS = []
ARR_PAYEE_WORDS = []
ARR_PAYEE_FIRST_NAME = []
ARR_PAYEE_MIDDLE_NAME = []
ARR_PAYEE_LAST_NAME = []

ARR_RIGHT_SECTION_CONTOURS = []
ARR_DATE_CONTOURS = []
ARR_AMOUNT_CONTOURS = []

# Helper Functions


In [3]:
def find_contour_with_least_area(contours):
    '''
    This function will calculate contours area and return the least one
    Parameters:
    contours: All contours hierarchy array
    '''
    
    # Step1: Find areas of every contour
    areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        # All areas appended in sequence
        areas.append(int(area))
        
    # Step2: Find the contour index with the least area
    least_area = areas[0]
    index = 0
    
    for i in range(0, len(areas)):
        if areas[i] <= least_area:
            # Update least value
            least_area = areas[i]
            index = i
            
    # Step3: Return the least area contour index
    return index

def find_contour_with_greatest_area(contours):
    '''
    This function will calculate contours area and return the least one
    Parameters:
    contours: All contours hierarchy array
    '''
    
    # Step1: Find areas of every contour
    areas = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        # All areas appended in sequence
        areas.append(int(area))
        
    # Step2: Find the contour index with the least area
    greatest_area = areas[0]
    index = 0
    
    for i in range(0, len(areas)):
        if areas[i] >= greatest_area:
            # Update least value
            greatest_area = areas[i]
            index = i
            
    # Step3: Return the least area contour index
    return index, areas


def draw_rectangle_on_img_contours_and_save_them(contours, image, arr_contours):
    '''
    This function will draw rectangles on image
    Parameters:
    contours : array
    image: image
    arr_contours: array in which cropped contours to be saved
    '''
    total_contours = len(contours)
    rect_color = (0,0,0)
    rect_stroke_width = 0
    
    for i in range(total_contours):
        cnt = contours[i]
        x,y,w,h = cv2.boundingRect(cnt)
        image = cv2.rectangle(image, (x,y), (x+w,y+h), rect_color, rect_stroke_width)
        
        # Cropping individual contours and saving them as separate image
        cropping_rectangles_and_saving_them(x,y,x+w,y+h,image,i,arr_contours)
        
    # To visualize all contours on image
    #plt.imshow(image)
    #plt.show()
    
    
def cropping_rectangles_and_saving_them(left,upper,right,lower, image_name, cropped_image_name, arr_contours):
    '''
    This function will crop all the bounding rectangles from an image
    Parameters:
    left=x,upper=y,right=x+w,lower=y+h
    image_name: original image
    cropped_image_name: individual cropped images name
    '''
    # Opens image using PIL
    im = Image.fromarray(image_name)

    # Crop image from original image
    box = (left,upper,right,lower)
    cropped_image = im.crop(box)

    # Appending cropped contours
    arr_contours.append(cropped_image)
    
def remove_contours_beyond_desired_area_range(contours, desired_area):
    '''
    This function will return only those contours that lie into our desired area range
    Parameters:
    contours: All contours hierarchy array
    '''

    # Step1: Find areas of every contour
    arr_cnts = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if int(area) >= desired_area:
            arr_cnts.append(cnt)

    return arr_cnts


def ret_x_cord_contour(contours):
    '''
    This func will get x-cord for contour
    Parameters:
    contours: Array containing all contours
    '''
    if int(cv2.contourArea(contours)) > 0:
        cent_moment = cv2.moments(contours)
        return cent_moment['m10']/cent_moment['m00']
    else:
        pass
    
def draw_rectangle_on_img_contours_and_croppping_them(contours, image, arr_contours):
    '''
    This function will draw rectangles on image
    Parameters:
    contours : array
    image: image
    arr_contours: array in which cropped contours to be saved
    '''
    total_contours = len(contours)
    rect_color = (255,0,0)
    rect_stroke_width = 2

    new_img = image.copy()

    for i in range(total_contours):
        cnt = contours[i]
        x,y,w,h = cv2.boundingRect(cnt)
        image = cv2.rectangle(image, (x,y), (x+w,y+h), rect_color, rect_stroke_width)

        # Step 5: Cropping individual contours and saving them in array
        cropping_rectangles_and_saving_them(x,y,x+w,y+h,new_img,i,arr_contours)
        
        
def get_all_childs_of_parent(parent, hierarchy):
    '''
    This function will return all the child of that contour
    Parameters:
    parent : Int value
    hierarchy: contours hierarchy
    '''
    # Initialize
    hier_len = hierarchy.shape[1] #Total contours
    child_contours = {} #Empty Dict

    for x in range(hier_len):
        # To loop through whole hierarchy
        next_contour, pre_contour, first_child, parent_contour = hierarchy[0][x]

        # If desired parent found, so append it all childs
        if parent == parent_contour:
            child_contours[x] = hierarchy[0][x]

    # return all the found contours
    return child_contours


def remove_extraneous_contours(index_to_keep, contours_dict):
    '''
    This function will remove extraneous contours from all child contours
    Parameters:
    index_to_keep : Array containing hierarchy values to keep
    contours_dict: All child contours dict
    '''
    # Initialize an empty dict
    final_dict = {}

    # Loop through all contours child dict and search for index_to_keep keys
    for x in index_to_keep:
        final_dict[x] = contours_dict[x]

    # Return only desired childs
    return final_dict

def remove_extraneous_contours_childs(contours,contours_dict):
    '''
    This function will remove extraneous contours from all contours
    Parameters:
    contours: Array containing all contours
    contours_dict: All child contours dict which we want to keep
    '''
    arr_keys = []
    new_contours = []

    # Appending all keys in a list
    for key,value in contours_dict.items():
        arr_keys.append(key)

    # Looping through all contours and keeping arr_keys only
    for i in range(len(contours)):
        for j in range(len(arr_keys)):
            if i == arr_keys[j]:
                new_contours.append(contours[i])

    #return new_contours
    return new_contours


def do_cheque_contrast_and_find_contours(img, contrast, brightness, dilation_factor, contour_mode, isComplete, thresh_new):
    if isComplete == True:
        #1 - 
        alpha = contrast # Contrast control (1.0-3.0)
        beta = brightness # Brightness control (0-100)
        adjusted = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

        #2 - convert the image to grayscale format
        img_gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)

        #3 - apply thresholding
        ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)

        #4 - Finding rows/col
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (dilation_factor[0],dilation_factor[1]))
        dilation = cv2.dilate(thresh, rect_kernel, iterations=1)

        #5- Find contours
        contours, hierarchy = cv2.findContours(image=dilation, mode=contour_mode, method=cv2.CHAIN_APPROX_NONE)
        image_copy = img.copy()
        cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=5, lineType=cv2.LINE_AA)

        #6- Cropped contours
        arr_contours = []
        draw_rectangle_on_img_contours_and_save_them(contours, thresh, arr_contours)

        #7- Return contours
        return arr_contours
    
    else:
        #4 - Finding rows/col
        thresh_new = np.array(thresh_new)
        rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (dilation_factor[0],dilation_factor[1]))
        dilation = cv2.dilate(thresh_new, rect_kernel, iterations=1)

        #5- Find contours
        contours, hierarchy = cv2.findContours(image=dilation, mode=contour_mode, method=cv2.CHAIN_APPROX_NONE)
        image_copy = img.copy()
        cv2.drawContours(image=image_copy, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=5, lineType=cv2.LINE_AA)

        #6- Find only desired area contour and sort them left->right
        new_contours = []

        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 10000 and area <100000:
                new_contours.append(cnt)
        
        contours = new_contours
        contours = sorted(contours, key=ret_x_cord_contour, reverse=False)
        
        #6- Cropped contours
        arr_contours = []
        draw_rectangle_on_img_contours_and_save_them(contours, thresh_new, arr_contours)

        #7- Return contours
        return arr_contours
    

def get_chars_from_word_img(binary_img):
    #1- make copy
    img = binary_img.copy()

    #2- Do padding
    img = cv2.copyMakeBorder(np.array(img), 30, 30, 0, 0, cv2.BORDER_CONSTANT, None, value=(0,0,0))

    #3- Find contours
    contours, hierarchy = cv2.findContours(image=img, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

    #4- Sort contours
    contours = sorted(contours, key=ret_x_cord_contour, reverse=False)

    #5- Crop contours
    arr = []
    draw_rectangle_on_img_contours_and_croppping_them(contours,img.copy(),arr)

    return arr

                
def get_cheque_right_section_rows(right_img):
    #1- Do contrast
    alpha = 1.5 # Contrast control (1.0-3.0)
    beta = 90 # Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(right_img.copy(), alpha=alpha, beta=beta)
    
    #2- convert the image to grayscale format
    img_gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)
    
    #3- apply thresholding
    ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)
    
    #4-  Finding rows
    rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (500,10))
    dilation = cv2.dilate(thresh, rect_kernel, iterations=1)
    
    #5- detect the contours
    contours, hierarchy = cv2.findContours(image=dilation, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
    #print("==>",type(contours))
    print(contours[0])
    if len(contours)>2:
        del contours[0]

    #6- Crop detected contours
    arr = []
    draw_rectangle_on_img_contours_and_save_them(contours, right_img.copy(), arr)
    return arr


def get_date_contours(date_img):
    #1- Do contrast
    alpha = 3.0 # Contrast control (1.0-3.0)
    beta = 100 # Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(date_img, alpha=alpha, beta=beta)

    #2- convert the image to grayscale format
    img_gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)

    #3- apply thresholding
    ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)

    #4- detect the contours
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

    #5- Remove Extraneous contours and sort contours
    #contours_to_keep_dict = get_all_childs_of_parent(1,hierarchy)
    #contours = remove_extraneous_contours_childs(contours,contours_to_keep_dict)
    #contours = sorted(contours, key=ret_x_cord_contour, reverse=False)

    #6- Crop detected contours
    arr = []
    draw_rectangle_on_img_contours_and_save_them(contours, thresh.copy(), arr)
    return arr


def get_amount_contours(amount_img):
    #1 - 
    y = amount_img.height #Height
    x = amount_img.width #Width

    #2- Crop
    amount_img = np.array(amount_img)
    amount_img = amount_img[40:y-15, 350:x-120]

    #3
    alpha = 3.0 # Contrast control (1.0-3.0)
    beta = 100 # Brightness control (0-100)
    adjusted = cv2.convertScaleAbs(amount_img, alpha=alpha, beta=beta)

    #4 - Grayscale
    img_gray = cv2.cvtColor(adjusted, cv2.COLOR_BGR2GRAY)

    #5- apply thresholding
    ret, thresh = cv2.threshold(img_gray, 150, 255, cv2.THRESH_OTSU|cv2.THRESH_BINARY_INV)

    #6- detect the contours
    contours, hierarchy = cv2.findContours(image=thresh, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)

    #7- Remove Extraneous contours and sort contours
    contours = remove_contours_beyond_desired_area_range(contours,1000)
    contours = sorted(contours, key=ret_x_cord_contour, reverse=False)

    #8- Crop detected contours
    arr = []
    draw_rectangle_on_img_contours_and_croppping_them(contours,thresh.copy(),arr)
    return arr

def resizing_cropped_img_to_28_by_28(arr_cropped_images, arr_new):
    
    for i in range(len(arr_cropped_images)):
        # Step:6 Calculating new size
        img_height = arr_cropped_images[i].height
        img_width = arr_cropped_images[i].width

        #print(img_height,img_width)

        # Tall and narrow image
        if img_height > img_width:
            img_height = 20
            scale_factor = arr_cropped_images[i].height / img_height  #old_height/new_height
            img_width = int(img_width/scale_factor)

        # Short and narrow image
        else:
            img_width = 20
            scale_factor = arr_cropped_images[i].width / img_width  #old_widht/new_width
            img_height = int(img_height/scale_factor)

        new_size = (img_width,img_height)
        img_resized = cv2.resize(np.array(arr_cropped_images[i]),new_size,0,0,cv2.INTER_AREA) #src,dest,dsize,0,0,cv2.INTER_AREA

        LEFT = math.ceil(4+(20-img_width)/2)
        RIGHT = math.floor(4+(20-img_width)/2)
        TOP = math.ceil(4+(20-img_height)/2)
        BOTTOM = math.ceil(4+(20-img_height)/2)

        img_28_by_28 = cv2.copyMakeBorder(img_resized, TOP, BOTTOM, LEFT, RIGHT, cv2.BORDER_CONSTANT, None, (0,0,0))
        arr_new.append(img_28_by_28)
        #plt.imshow(img_28_by_28)
        #plt.show()



# Image Processing

In [4]:
def recognize_cheque_details(image):
    global ARR_LEFT_SECTION_CONTOURS
    global ARR_PAYEE_CONTOURS
    global ARR_PAYEE_WORDS
    global ARR_PAYEE_FIRST_NAME
    global ARR_PAYEE_MIDDLE_NAME
    global ARR_PAYEE_LAST_NAME

    global ARR_RIGHT_SECTION_CONTOURS
    global ARR_DATE_CONTOURS
    global ARR_AMOUNT_CONTOURS
    
    first_name_prediction = ""
    middle_name_prediction = ""
    last_name_prediction = ""
    # ------------------------------------------------
    # STEP 1: SPLIT IMAGE BY 30 AND 70 RATIO
    # ------------------------------------------------
    img = cv2.imread(image)
    
    y = img.shape[0] #Height
    x = img.shape[1] #Width
    
    crop_ratio = int(x/3) # 100%-30% = 70%
    
    # Left section 70%
    IMG_LEFT_CROP = img[0:y, 0:x-crop_ratio] #[height, width]
    
    # Right section 30%
    IMG_RIGHT_CROP = img[0:y, x-crop_ratio:x] #[height, width] 
    
    
    # ------------------------------------------------
    # STEP 2: EXTRACT ONLY DESIRED SECTION FROM LEFT
    #         CROP IMAGE
    # ------------------------------------------------
    # Vertically 20-25% above and the below area is of our no use. Discard it.
    y = IMG_LEFT_CROP.shape[0] #Height
    x = IMG_LEFT_CROP.shape[1] #Width
    
    crop_ratio = int(y/1.35) # 100%-13.5% = 86.5%
    IMG_LEFT_CROP = IMG_LEFT_CROP[y-crop_ratio:y-(y-crop_ratio), 0:x] #[height, width] => [362,]
    
    # Cropping more from bottom
    image_copy = IMG_LEFT_CROP.copy()
    IMG_LEFT_CROP = image_copy[0:y-150,0:x]
    
    # ------------------------------------------------
    # STEP 2.1: FIND CONTOURS ON LEFT SECTION
    # ------------------------------------------------
    ARR_LEFT_SECTION_CONTOURS = do_cheque_contrast_and_find_contours(IMG_LEFT_CROP, 3, 150, (500,10), cv2.RETR_EXTERNAL, True, None)
    ARR_LEFT_SECTION_CONTOURS.reverse()
    
    # ------------------------------------------------
    # STEP 2.2: EXTRACTING PAYEE NAME
    # ------------------------------------------------
    ARR_PAYEE_CONTOURS.clear()
    ARR_PAYEE_CONTOURS = do_cheque_contrast_and_find_contours(IMG_LEFT_CROP, 3, 150, (70,1), cv2.RETR_LIST, False, ARR_LEFT_SECTION_CONTOURS[0].copy())
    
    if len(ARR_PAYEE_CONTOURS) == 1:
        ARR_PAYEE_FIRST_NAME.clear()
        ARR_PAYEE_FIRST_NAME = get_chars_from_word_img(ARR_PAYEE_CONTOURS[0])
    elif len(ARR_PAYEE_CONTOURS) == 2:
        ARR_PAYEE_FIRST_NAME.clear()
        ARR_PAYEE_FIRST_NAME = get_chars_from_word_img(ARR_PAYEE_CONTOURS[0])
        ARR_PAYEE_LAST_NAME.clear()
        ARR_PAYEE_LAST_NAME = get_chars_from_word_img(ARR_PAYEE_CONTOURS[1])
    elif len(ARR_PAYEE_CONTOURS) == 3:
        ARR_PAYEE_FIRST_NAME.clear()
        ARR_PAYEE_FIRST_NAME = get_chars_from_word_img(ARR_PAYEE_CONTOURS[0])
        ARR_PAYEE_MIDDLE_NAME.clear()
        ARR_PAYEE_MIDDLE_NAME = get_chars_from_word_img(ARR_PAYEE_CONTOURS[1])
        ARR_PAYEE_LAST_NAME.clear()
        ARR_PAYEE_LAST_NAME = get_chars_from_word_img(ARR_PAYEE_CONTOURS[2])
    #else:
        # Do nothing
        
     
    # ------------------------------------------------
    # STEP 3: EXTRACT ONLY DESIRED SECTION FROM RIGHT
    #         CROP IMAGE
    # -----------------------------------------------
    y = IMG_RIGHT_CROP.shape[0] #Height
    x = IMG_RIGHT_CROP.shape[1] #Width
    
    bottom_crop_ratio = int(y/2.5) # 100%-25% = 75%
    top_crop_ratio = int(y/6.8) # 100%-68% = 30%

    IMG_RIGHT_CROP = IMG_RIGHT_CROP[top_crop_ratio:y-bottom_crop_ratio, 0:x] #[height, width] => [362,]
    ARR_RIGHT_SECTION_CONTOURS = get_cheque_right_section_rows(IMG_RIGHT_CROP)
    
    
    if len(ARR_RIGHT_SECTION_CONTOURS) == 2:
        ARR_RIGHT_SECTION_CONTOURS.reverse()
    
        # ------------------------------------------------
        # STEP 3.1: FIND DATE CONTOURS
        # -----------------------------------------------
        date_img = np.array(ARR_RIGHT_SECTION_CONTOURS[0])
        ARR_DATE_CONTOURS = get_date_contours(date_img)
        ARR_DATE_CONTOURS.reverse()
        if len(ARR_DATE_CONTOURS) > 8:
            del ARR_DATE_CONTOURS[0]
            
        # ------------------------------------------------
        # STEP 3.2: FIND AMOUNT CONTOURS
        # -----------------------------------------------
            
        y = ARR_RIGHT_SECTION_CONTOURS[1].height #Height
        x = ARR_RIGHT_SECTION_CONTOURS[1].width #Width
        
        image_copy = ARR_RIGHT_SECTION_CONTOURS[1].copy()
        image_copy = np.array(image_copy)
        
        crop_amount_in_words = image_copy[40:y-15, 350:x-120]
        
        ARR_AMOUNT_CONTOURS = get_amount_contours(ARR_RIGHT_SECTION_CONTOURS[1])
        
    # ------------------------------------------------
    # STEP 4: CONVERT ALL EXTRACTED CONTOURS TO
    #         28X28
    # -----------------------------------------------
    arr_amount_28x28 = []
    resizing_cropped_img_to_28_by_28(ARR_AMOUNT_CONTOURS, arr_amount_28x28)
    
    arr_date_28x28 = []
    resizing_cropped_img_to_28_by_28(ARR_DATE_CONTOURS, arr_date_28x28)
    
    arr_payee_first_name_28x28 = []
    resizing_cropped_img_to_28_by_28(ARR_PAYEE_FIRST_NAME, arr_payee_first_name_28x28)
    
    arr_payee_middle_name_28x28 = []
    resizing_cropped_img_to_28_by_28(ARR_PAYEE_MIDDLE_NAME, arr_payee_middle_name_28x28)
    
    arr_payee_last_name_28x28 = []
    resizing_cropped_img_to_28_by_28(ARR_PAYEE_LAST_NAME, arr_payee_last_name_28x28)
    
    # ------------------------------------------------
    # STEP 5: FLATTEN ALL 28X28 IMAGES
    # ------------------------------------------------
    x_arr_amount = []
    for i in range(len(arr_amount_28x28)):
        x_arr_amount.append(arr_amount_28x28[i].ravel())
        
    x_arr_date = []
    for i in range(len(arr_date_28x28)):
        x_arr_date.append(arr_date_28x28[i].ravel())
        
    x_arr_first_name = []
    for i in range(len(arr_payee_first_name_28x28)):
        x_arr_first_name.append(arr_payee_first_name_28x28[i].ravel())
        
    x_arr_middle_name = []
    for i in range(len(arr_payee_middle_name_28x28)):
        x_arr_middle_name.append(arr_payee_middle_name_28x28[i].ravel())
        
    x_arr_last_name = []
    for i in range(len(arr_payee_last_name_28x28)):
        x_arr_last_name.append(arr_payee_last_name_28x28[i].ravel())
        
    # ------------------------------------------------
    # STEP 6: LOAD DIGITS MODEL AND CREATE SESSION
    # ------------------------------------------------
    # Create a graph obj placeholder
    graph = tf.Graph()

    # Creating a sess obj and linking session and the graph
    sess = tf.compat.v1.Session(graph=graph)

    # Loading the Model
    load(sess=sess, tags=[tag_constants.SERVING], export_dir='SavedModel')
    
    # ------------------------------------------------
    # STEP 7: MAKING DIGITS PREDICTIONS
    # ------------------------------------------------
    # Return the `Tensor` with the given `name` and index=0 result
    X = graph.get_tensor_by_name('X:0')

    # Get hold of the tensor that will hold the predictions from the graph. Store these under y_pred
    y_pred = graph.get_tensor_by_name('accuracy_calc/prediction:0')

    # fetches = y_pred(the output we are after)
    amount_prediction = sess.run(fetches=y_pred, feed_dict={X:x_arr_amount})
    date_prediction = sess.run(fetches=y_pred, feed_dict={X:x_arr_date})
    
    # ------------------------------------------------
    # STEP 8: LOAD ALPHABETS MODEL AND CREATE SESSION
    # ------------------------------------------------
    # Create a graph obj placeholder
    graph = tf.Graph()

    # Creating a sess obj and linking session and the graph
    sess = tf.compat.v1.Session(graph=graph)

    # Loading the Model
    load(sess=sess, tags=[tag_constants.SERVING], export_dir='SavedModel_Alpha')
    
    # ------------------------------------------------
    # STEP 9: MAKING DIGITS PREDICTIONS
    # ------------------------------------------------
    # Return the `Tensor` with the given `name` and index=0 result
    X = graph.get_tensor_by_name('X:0')

    # Get hold of the tensor that will hold the predictions from the graph. Store these under y_pred
    y_pred = graph.get_tensor_by_name('accuracy_calc/prediction:0')

    # fetches = y_pred(the output we are after)
    if len(x_arr_first_name) > 0:
        first_name_prediction = sess.run(fetches=y_pred, feed_dict={X:x_arr_first_name})
    if len(x_arr_middle_name) > 0:
        middle_name_prediction = sess.run(fetches=y_pred, feed_dict={X:x_arr_middle_name})
    if len(x_arr_last_name) > 0:
        last_name_prediction = sess.run(fetches=y_pred, feed_dict={X:x_arr_last_name})
        
    # ------------------------------------------------
    # STEP 10: DECODING ALPHABETS PREDICTIONS
    # ------------------------------------------------
    arr_predicted_first_name = []
    arr_predicted_middle_name = []
    arr_predicted_last_name = []
    
    for i in range(len(first_name_prediction)):
        for key,value in ALPHABETS.items():
            if first_name_prediction[i] == key:
                arr_predicted_first_name.append(value)
                
    for i in range(len(middle_name_prediction)):
        for key,value in ALPHABETS.items():
            if middle_name_prediction[i] == key:
                arr_predicted_middle_name.append(value)
                
    for i in range(len(last_name_prediction)):
        for key,value in ALPHABETS.items():
            if last_name_prediction[i] == key:
                arr_predicted_last_name.append(value)
                
    # ------------------------------------------------
    # STEP 11: CONVERTING ARR INTO STRING
    # ------------------------------------------------
    amount = ''.join(map(str,amount_prediction))
    amount = 'Rs.' + amount
    date = ''.join(map(str,date_prediction))
    payee_fn = ''.join(map(str,arr_predicted_first_name))
    payee_mn = ''.join(map(str,arr_predicted_middle_name))
    payee_ln = ''.join(map(str,arr_predicted_last_name))
    
    new_date = ''
    for i in range(len(date)):
        if i == 2 or i == 4:
            new_date += '/' + date[i]
        else:
            new_date += date[i]
    
    print(amount)
    print(new_date)
    print(payee_fn)
    print(payee_mn)
    print(payee_ln)

In [5]:
recognize_cheque_details(SAMPLE_IMAGE)

[[[503 619]]

 [[502 620]]

 [[501 620]]

 ...

 [[506 619]]

 [[505 619]]

 [[504 619]]]
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from SavedModel/variables/variables
INFO:tensorflow:Restoring parameters from SavedModel_Alpha/variables/variables
Rs.430228
31/21/1988
EAYQUE

ALY
